# Segmenting and Clustering Neighborhoods in Toronto

#### Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe

Durch pd.read_html() kann eine Tabelle von auf einer Webseite in einen Pandas Dataframe eingelesen werden

In [43]:
import pandas as pd
import numpy as np

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
df = pd.read_html(url, header=0)[0]

#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
df=df[df['Borough']!='Not assigned']
df.head

<bound method NDFrame.head of     Postcode           Borough  \
2        M3A        North York   
3        M4A        North York   
4        M5A  Downtown Toronto   
5        M5A  Downtown Toronto   
6        M6A        North York   
7        M6A        North York   
8        M7A      Queen's Park   
10       M9A         Etobicoke   
11       M1B       Scarborough   
12       M1B       Scarborough   
14       M3B        North York   
15       M4B         East York   
16       M4B         East York   
17       M5B  Downtown Toronto   
18       M5B  Downtown Toronto   
19       M6B        North York   
22       M9B         Etobicoke   
23       M9B         Etobicoke   
24       M9B         Etobicoke   
25       M9B         Etobicoke   
26       M9B         Etobicoke   
27       M1C       Scarborough   
28       M1C       Scarborough   
29       M1C       Scarborough   
31       M3C        North York   
32       M3C        North York   
33       M4C         East York   
34       M5C  Down

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [6]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [7]:
df.head()

Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae

#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [8]:
df.shape

(103, 3)

# get the latitude and the longitude coordinates of each neighborhood

In [9]:
import geocoder

In [10]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M4G')

[43.70976500000006, -79.36390090899994]

In [11]:
postal_codes = df['Postcode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [12]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [13]:
df.head()

Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.811525   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.785730   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.765690   
3      M1G  Scarborough                                  Woburn  43.768359   
4      M1H  Scarborough                               Cedarbrae  43.769688   

   Longitude  
0 -79.195517  
1 -79.158750  
2 -79.175256  
3 -79.217590  
4 -79.239440

# Question 3 

### Get the Location Data of Toronto and creating a Map showing the Boroughs and Neighbourhoods

In [14]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium


In [15]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

WARNING conda.base.context:use_only_tar_bz2(632): Conda is constrained to only using the old .tar.bz2 file format because you have conda-build installed, and it is <3.18.3.  Update or remove conda-build to get smaller downloads and faster extractions.
Solving environment: done

# All requested packages already installed.



In [16]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Creating a map only showing the bouroughs that include Toronto in their name

In [18]:
df_toronto = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto.head()

Postcode          Borough                   Neighbourhood   Latitude  \
0      M4E     East Toronto                     The Beaches  43.676845   
1      M4K     East Toronto    The Danforth West, Riverdale  43.683262   
2      M4L     East Toronto  The Beaches West, India Bazaar  43.667965   
3      M4M     East Toronto                 Studio District  43.662766   
4      M4N  Central Toronto                   Lawrence Park  43.728160   

   Longitude  
0 -79.295225  
1 -79.355120  
2 -79.314673  
3 -79.334830  
4 -79.387085

In [19]:
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

#### Define Foursquare Credentials and Version

In [20]:
CLIENT_ID = '1GYMO4W3YGVISGOPA5T2DC4VTZQ3LKGNYNWKG2H103TIWIFK' # your Foursquare ID
CLIENT_SECRET = 'I4CCAVGXLHIF5R0YGAQWFY0RVSJTTWHFFU2YCHZSDEXAUBER' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

## Explore Neighborhoods in Toronto

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [22]:
import requests
from pandas.io.json import json_normalize

In [23]:
LIMIT = 100 #Limit it to 100 Venues per Borough
radius = 500

In [36]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [37]:
toronto_venues.head()

Neighbourhood  Neighborhood Latitude  Neighborhood Longitude  \
0   The Beaches              43.676845              -79.295225   
1   The Beaches              43.676845              -79.295225   
2   The Beaches              43.676845              -79.295225   
3   The Beaches              43.676845              -79.295225   
4   The Beaches              43.676845              -79.295225   

                                Venue  Venue Latitude  Venue Longitude  \
0  The Big Carrot Natural Food Market       43.678879       -79.297734   
1                   Glen Manor Ravine       43.676821       -79.293942   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                 Glen Stewart Ravine       43.676300       -79.294784   
4                       Upper Beaches       43.680563       -79.292869   

         Venue Category  
0     Health Food Store  
1                 Trail  
2                   Pub  
3  Other Great Outdoors  
4          Neighborhood

### Analyze the Neighbourhoods

In [38]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Neighbourhood  Adult Boutique  Afghan Restaurant  American Restaurant  \
0   The Beaches               0                  0                    0   
1   The Beaches               0                  0                    0   
2   The Beaches               0                  0                    0   
3   The Beaches               0                  0                    0   
4   The Beaches               0                  0                    0   

   Antique Shop  Art Gallery  Art Museum  Arts & Crafts Store  \
0             0            0           0                    0   
1             0            0           0                    0   
2             0            0           0                    0   
3             0            0           0                    0   
4             0            0           0                    0   

   Asian Restaurant  Athletics & Sports  ...  Toy / Game Store  Trail  \
0                 0                   0  ...                 0      0   
1                 0                   0  ...                 0      1   
2                 0                   0  ...                 0      0   
3                 0                   0  ...                 0      0   
4                 0                   0  ...                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Wings Joint  Women's Store  Yoga Studio  
0                      0         0            0              0            0  
1                      0         0            0              0            0  
2                      0         0            0              0            0  
3                      0         0            0              0            0  
4                      0         0            0              0            0  

[5 rows x 213 columns]

In [34]:
toronto_onehot.shape

(1737, 212)

#### group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [39]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

Neighbourhood  Adult Boutique  \
0                            Adelaide, King, Richmond        0.000000   
1                                         Berczy Park        0.000000   
2        Brockton, Exhibition Place, Parkdale Village        0.000000   
3   Business Reply Mail Processing Centre 969 Eastern        0.000000   
4   CN Tower, Bathurst Quay, Island airport, Harbo...        0.000000   
5                         Cabbagetown, St. James Town        0.000000   
6                                  Central Bay Street        0.000000   
7           Chinatown, Grange Park, Kensington Market        0.000000   
8                                            Christie        0.000000   
9                                Church and Wellesley        0.012195   
10                     Commerce Court, Victoria Hotel        0.000000   
11                                         Davisville        0.000000   
12                                   Davisville North        0.000000   
13  Deer Park, Forest Hill SE, Rathnelly, South Hi...        0.000000   
14           Design Exchange, Toronto Dominion Centre        0.000000   
15                       Dovercourt Village, Dufferin        0.000000   
16             First Canadian Place, Underground city        0.000000   
17                Forest Hill North, Forest Hill West        0.000000   
18                     Harbord, University of Toronto        0.000000   
19  Harbourfront East, Toronto Islands, Union Station        0.000000   
20                          Harbourfront, Regent Park        0.000000   
21                      High Park, The Junction South        0.000000   
22                                      Lawrence Park        0.000000   
23                           Little Portugal, Trinity        0.000000   
24                        Moore Park, Summerhill East        0.000000   
25                                 North Toronto West        0.000000   
26                             Parkdale, Roncesvalles        0.000000   
27                                           Rosedale        0.000000   
28                                 Runnymede, Swansea        0.000000   
29                           Ryerson, Garden District        0.000000   
30                                     St. James Town        0.000000   
31                    Stn A PO Boxes 25 The Esplanade        0.000000   
32                                    Studio District        0.000000   
33                The Annex, North Midtown, Yorkville        0.000000   
34                                        The Beaches        0.000000   
35                     The Beaches West, India Bazaar        0.000000   
36                       The Danforth West, Riverdale        0.000000   

    Afghan Restaurant  American Restaurant  Antique Shop  Art Gallery  \
0            0.000000             0.030000      0.000000     0.010000   
1            0.000000             0.000000      0.000000     0.015873   
2            0.000000             0.000000      0.000000     0.014925   
3            0.000000             0.030000      0.000000     0.010000   
4            0.000000             0.000000      0.000000     0.000000   
5            0.000000             0.000000      0.000000     0.000000   
6            0.000000             0.010000      0.000000     0.010000   
7            0.000000             0.000000      0.000000     0.010638   
8            0.000000             0.000000      0.000000     0.000000   
9            0.012195             0.012195      0.000000     0.000000   
10           0.000000             0.030000      0.000000     0.010000   
11           0.000000             0.000000      0.000000     0.000000   
12           0.000000             0.000000      0.000000     0.000000   
13           0.000000             0.000000      0.000000     0.000000   
14           0.000000             0.030000      0.000000     0.010000   
15           0.000000             0.000000      0.000000     0.000000   
16           0.000000   

#### print each neighborhood along with the top 5 most common venues

In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.07
2                 Café  0.06
3            Gastropub  0.03
4  American Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.11
1        Cocktail Bar  0.05
2          Restaurant  0.05
3              Bakery  0.03
4  Seafood Restaurant  0.03


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1                    Café  0.06
2  Furniture / Home Store  0.06
3          Sandwich Place  0.04
4                     Bar  0.04


----Business Reply Mail Processing Centre 969 Eastern----
         venue  freq
0  Coffee Shop  0.07
1          Bar  0.04
2         Café  0.04
3        Hotel  0.04
4   Steakhouse  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0         Coffee Shop  0.

#### create a Pandas DF out of this info

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

create a DF with the 10 top venues

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighbourhood 1st Most Common Venue  \
0                           Adelaide, King, Richmond                 Hotel   
1                                        Berczy Park           Coffee Shop   
2       Brockton, Exhibition Place, Parkdale Village           Coffee Shop   
3  Business Reply Mail Processing Centre 969 Eastern           Coffee Shop   
4  CN Tower, Bathurst Quay, Island airport, Harbo...           Coffee Shop   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0             Coffee Shop                  Café   Japanese Restaurant   
1            Cocktail Bar            Restaurant    Italian Restaurant   
2  Furniture / Home Store                  Café            Restaurant   
3                   Hotel                  Café            Steakhouse   
4      Italian Restaurant                   Bar            Restaurant   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
0            Restaurant          Burger Joint        Breakfast Spot   
1            Steakhouse                  Café              Beer Bar   
2        Sandwich Place                   Bar                 Hotel   
3                   Bar    Italian Restaurant   American Restaurant   
4  Gym / Fitness Center                  Café        Sandwich Place   

  8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
0            Steakhouse         Deli / Bodega                 Bakery  
1    Seafood Restaurant           Cheese Shop                 Bakery  
2                Bakery    Italian Restaurant            Supermarket  
3                   Pub           Pizza Place       Sushi Restaurant  
4             Speakeasy                  Park                    Pub

## Cluster Neighborhoods

In [58]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [59]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [61]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

manhattan_merged.head() # check the last columns!

ValueError: cannot insert Cluster Labels, already exists

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters